In [41]:
import random
import csv
import math
import sys
from pathlib import PurePath
from pathlib import Path

arg1 = '/content/drive/MyDrive/Abf1_all_sf_MACS3_peaks_dyads.bed'
arg2 = 'all_sf'
arg3 = 4
arg4 = [10, 20, 50, 100]
arg5 = 'FALSE'

all_dyads = arg1 #input path for a BED of all MACS3 dyads, need to add a broad flag and condition to adjust range
ptype = arg2 # a short label for the output dyads
max_dy = int(arg3) #max number of dyads to search for per entry
max_list = list(arg4) #list of maximum footprint sizes to check
codes = arg5 # print exit codes for each dyad to terminal

def sort_key(line): # A key to feed to the sort function that acts like bash sort
	fields = line.split()
	try:
		return fields[0], int(fields[1]), int(fields[2])
	except (IndexError, ValueError):
		return () # sort invalid lines together

dir_parts = PurePath(all_dyads).parts
filename = dir_parts[-1]
srtfile = ('_').join([filename[0:3], ptype, 'dyads_sorted.bed'])
srtout = '/content/drive/MyDrive/Abf1_all_sf_MACS3_peaks_dyads_sorted.bed' #str(dir_parts[0] + '/merging_analysis/raw_dyads/' + srtfile)

with open(all_dyads) as inlines:
  lines = inlines.readlines()
  inlines.close()

lines.sort(key=sort_key)

with open(srtout, 'w') as sortfile:
  sortfile.writelines(lines)
  sortfile.close()

for max in max_list:
  mpname = ('_').join([dir_parts[0], ptype, str(max), 'sorted_merged.tsv'])
  mpout = str('/content/drive/MyDrive/Abf1_all_sf_alg_out_' + str(max) + '.tsv') #str(dir_parts[0] + '/merging_analysis/raw_dyads/' + mpname)
  print('Searching for clusters of at most ' + str(max_dy) + ' dyads')
  print('Limiting to total search area of ' + str(max) + 'bp')
  with open(srtout, 'r') as dyads:
    values = csv.reader(dyads, delimiter = '\t')
    a = list(values)
    mpset = open(mpout, 'w+')
    for i in range(1, len(a) - 1): # skip ends to avoid errors, should still capture all dyads
      dc = a[i] # current dyad
      chrc = dc[0] # chr number of current dyad
      db = a[i - 1] # dyad before current
      chrb = db[0]
      da = a[i + 1] # dyad after current
      chra = da[0]
      dm1 = int(dc[1]) # midpoint of entry
      dm2 = int(dc[2]) # +1 point of entry
      
      xc1 = 0 # reset extension counters for new entry - marks how far we've moved to find a neighbor dyad for dm1
      xc2 = 0 # for dm2
      xct = xc1 + xc2 # total region dyad has been expanded to so far
      k = 1 # set the dyad counter for new entry - if we are reading an entry, there is one dyad already
      
      if xct < int(max): # limit searching to the current max region size

        while k < (max_dy - 1): # limit searching to a number of dyads found, higher priority exit condition
          
          if chrc == chra and chrc == chrb: # all entries on same chromosome
            
            if dm1 <= int(db[2]) and dm2 >= int(da[1]): # overlap on both ends, define new dyad, record distance
              k = k + 2
              dm1 = math.floor(int(da[2]) - (((int(da[2])) - (int(db[1])))/2))
              dm2 = dm1 + 1
              xc1 = dm1 - int(db[1])
              xc2 = int(da[2]) - dm1

            elif dm1 <= int(db[2]) and dm2 < int(da[1]): # overlap on left end only, define new dyad, extend right side
              k = k + 1
              xc2 = xc2 + 1
              dm1 = math.floor(int(dm2) - (((int(dm2)) - (int(db[1])))/2))
              xc1 = dm1 - int(db[1])
              dm2 = dm1 + xc2
          
            elif dm1 > int(db[2]) and dm2 >= int(da[1]): # overlap on right end only, define new dyad, extend left side
              k = k + 1
              xc1 = xc1 + 1
              dm2 = math.floor(int(da[2]) - (((int(da[2])) - (int(dm1)))/2))
              xc2 = int(da[2]) - dm2
              dm1 = dm2 - xc1
          
            else: # no overlap, but we know neighbors are on same chr, so keep checking
              xc1 = xc1 + 1
              xc2 = xc2 + 1
              dm1 = dm2 - xc1
              dm2 = dm2 + xc2
              
          elif chrc == chrb or chrc == chra: #current entry on same chr as one neighbor
          
            if dm1 <= int(db[2]) and chrc != chra: # left match - don't add to right
              k = k + 1
              dm1 = math.floor(dm2 - ((dm2 - (int(db[1])))/2))
              xc1 = dm1 - int(da[2])
              xc2 = xc2
              dm2 = dm1 + 1
              
            elif dm2 >= int(da[1]) and chrc != chrb: # right match - don't add to left
              k = k + 1
              dm1 = math.floor(int(da[2]) - (((int(da[2]) - dm1))/2))
              dm2 = dm1 + 1
              xc1 = xc1
              xc2 = int(da[2]) - dm2
            
            else: #no overlap, but we know one neighbor could be matched still
              if chrc == chrb: #if previous entry is on chr
                xc1 = xc1 + 1
                dm2 = dm1
                dm1 = dm2 - xc1
                
              if chrc == chra: #if next entry is on chr
                xc2 = xc2 + 1
                dm1 = dm1
                dm2 = dm1 + xc2
        
          else: #no matches left on chromosome for current dyad
            print('No remaining matches for current entry - moving to next entry')
            break

          # Update all the values from the current iteration  
          dm1 = dm1
          dm2 = dm2
          xc1 = xc1
          xc2 = xc2
          xct = xc1 + xc2
          k = k
          
          if xct >= int(max): # check if max footprint size has been reached 
            break

      if codes == 'TRUE':
        if k >= max_dy and codes:
          print('Exit 1: Max number of requested dyads found - recording and moving to next entry') 

        elif k < max_dy:
          print('Exit 2: Cannot match dyads up to requested max for current entry - recording and moving to next entry')
        
        elif xct >= max:
          print('Exit 3: Maximum requested size of footprint reached - recording and moving to next entry')
      
        elif xct < max and k < max_dy:
          print('Exit 4: No dyads left on chr to match for current entry - recording and moving to next entry')
      
      # Record the dyad from the final set of dm values reported
      dm1 = math.floor(int(dm2) - (((int(dm2)) - (int(dm1)))/2))
      dm2 = dm1 + 1
      mpset.write(chrc)
      mpset.write('\t')
      mpset.write(str(dm1))
      mpset.write('\t')
      mpset.write(str(dm2))
      mpset.write('\t')
      mpset.write(str(filename[0:3] + '_' + str(ptype) + '_region_' + str(i)))
      mpset.write('\t')
      mpset.write(str(dc[3]))
      mpset.write('\t')
      mpset.write(str(xc1))
      mpset.write('\t')
      mpset.write(str(xc2))
      mpset.write('\t')
      mpset.write(str(k))
      mpset.write('\n')
mpset.close()

Searching for clusters of at most 4 dyads
Limiting to total search area of 10bp
Searching for clusters of at most 4 dyads
Limiting to total search area of 20bp
Searching for clusters of at most 4 dyads
Limiting to total search area of 50bp
Searching for clusters of at most 4 dyads
Limiting to total search area of 100bp
